This the exercise for week 1 of the MLOps zoomcamp

In [2]:
import pandas as pd
import pyarrow
import pickle
import os
import mlflow
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/zharec/MLops-zoomcamp-2023/Week_2/mlruns/1', creation_time=1685189956696, experiment_id='1', last_update_time=1685189956696, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(path):
    df = pd.read_parquet(path)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds()/60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ['PULocationID','DOLocationID']
    
    df[categorical] = df[categorical].astype(str)
    
    return df
    

In [5]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715206931833

In [20]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "zhare")
    
    mlflow.log_param("train-data-path", "Week_2/data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "Week_2/data/green_tripdata_2021-02.parquet")
    
    alpha = 0.1
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="model_pickle")
    
    

In [12]:
lr = Ridge(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.5095897790553074

In [17]:

if not os.path.exists('models'):
    os.mkdir('models')


In [18]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [26]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [23]:


def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50   
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        
    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_Rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42,
}

# best_result = fmin(
#     fn=objective,
#     space=search_space,
#     algo=tpe.suggest,
#     max_evals=50,
#     trials=Trials())

In [31]:
with mlflow.start_run():
    

    best_params = {
        'max_depth': 30,
        'learning_rate': 0.05184548781365737,
        'reg_alpha': 0.27015394779266766,
        'min_child_weight': 0.4099048192654753,
        'objective': 'reg:linear',
        'seed': 42,
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50   
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    with open("models/booster.bin", "wb") as f_out:
        pickle.dump(dv, f_out)
    
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact("models/booster.bin", artifact_path="model_pickle")
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")


[23:06:26] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:20.26442
[1]	validation-rmse:19.38857
[2]	validation-rmse:18.56276
[3]	validation-rmse:17.78351
[4]	validation-rmse:17.04922
[5]	validation-rmse:16.35769
[6]	validation-rmse:15.70642
[7]	validation-rmse:15.09442
[8]	validation-rmse:14.51869
[9]	validation-rmse:13.97733
[10]	validation-rmse:13.46870
[11]	validation-rmse:12.99181
[12]	validation-rmse:12.54452
[13]	validation-rmse:12.12538
[14]	validation-rmse:11.73306
[15]	validation-rmse:11.36657
[16]	validation-rmse:11.02388
[17]	validation-rmse:10.70482
[18]	validation-rmse:10.40657
[19]	validation-rmse:10.12881
[20]	validation-rmse:9.86969
[21]	validation-rmse:9.62910
[22]	validation-rmse:9.40410
[23]	validation-rmse:9.19593
[24]	validation-rmse:9.00315
[25]	validation-rmse:8.82293
[26]	validation-rmse:8